In [ ]:
def ODPol2Matrix(DfOD_Geometry):
    '''
        DfOD_Geometry: dataframe -> [SAMPN,PERNO,origin,destination,dep_time,origin_osmid,destination_osmid]
                                    |       ,   ,       ,           ,       ,              ,                |
        NOTE: DfOD_Geometry is od_demand file. (Created for each of the geometrical objects we will work) [grid,ring,hexagon,poygons]
        NOTE: DfOD_Geometry contains already all the informations about how many people leave the geometry. 
        NOTE: matrix_OD is a matrix whose entrances are indicised from 0 t0 Norigins. It makes easier to use u and v for this reason.

    '''
    if 'origin' not in DfOD_Geometry.columns:
        raise KeyError('Missing origin column from Geometric Plan Tessellation Object')
    if 'destination' not in DfOD_Geometry.columns:
        raise KeyError('Missing destination column from Geometric Plan Tessellation Object')         
    origins = np.unique(DfOD_Geometry['origin'].to_numpy())
    destinations = np.unique(DfOD_Geometry['destination'].to_numpy())
    mapOD = DfOD_Geometry.groupby('origin')['destination'].unique().apply(lambda x: x[0]).to_dict()
    zero2mapidxorigin = {i:idx for idx,i in enumerate(origins)}
    zero2mapidxdestination = {i:idx for idx,i in enumerate(destinations)}
    mapidx2origin = {idx:i for idx,i in enumerate(origins)}
    mapidx2destination = {idx:i for idx,i in enumerate(destinations)}
    dimensionality_matrix = (len(origins),len(destinations))
    matrix_OD = np.zeros(dimensionality_matrix)
    trips_OD = DfOD_Geometry.groupby(['origin']).count()['destination'].to_numpy(dtype= int)
    for idx in range(len(origins)):
        matrix_OD[zero2mapidxorigin[origins[idx]],zero2mapidxdestination[mapOD[origins[idx]]]] = trips_OD[idx]
    matrix_out = np.sum(matrix_OD,axis=1)
    matrix_in = np.sum(matrix_OD,axis=0)
    return matrix_OD,matrix_out,matrix_in,zero2mapidxorigin,zero2mapidxdestination,mapidx2origin,mapidx2destination




def OD2matrix(df_od):
    '''
        Create: 
            1) 02mapidxorigin {0:origin1,1:origin2,...}
            2) 02mapidxdestination {0:destination1,1:destination2,...}
            3) mapidx2origin {origin1:0,origin2:1,...}
            4) mapidx2destination {destination1:0,destination2:1,...}
            origin_i,destination_i are the nominative of self.gdf_polygons
    '''
    origins = np.unique(df_od['origin'].to_numpy())
    destinations = np.unique(df_od['destination'].to_numpy())
    mapOD = df_od.groupby('origin')['destination'].unique().apply(lambda x: x[0]).to_dict()
    zero2mapidxorigin = {i:idx for idx,i in enumerate(origins)}
    zero2mapidxdestination = {i:idx for idx,i in enumerate(destinations)}
    mapidx2origin = {idx:i for idx,i in enumerate(origins)}
    mapidx2destination = {idx:i for idx,i in enumerate(destinations)}
    dimensionality_matrix = (len(origins),len(destinations))
    matrix_OD = np.zeros(dimensionality_matrix)
    print('OD: ',np.shape(matrix_OD))
    trips_OD = df_od.groupby(['origin']).count()['destination'].to_numpy(dtype= int)
    print('Origin destination couples: ',np.shape(trips_OD))
    for idx in range(len(origins)):
            matrix_OD[zero2mapidxorigin[origins[idx]],zero2mapidxdestination[mapOD[origins[idx]]]] = trips_OD[idx]
    print('matrix OD:\n',matrix_OD)
    matrix_out = np.sum(matrix_OD,axis=1)
    print('matrix_out:\n',matrix_out)
    matrix_in = np.sum(matrix_OD,axis=0)
    print('matrix_in:\n',matrix_in)
    return matrix_OD,matrix_out,matrix_in,zero2mapidxorigin,zero2mapidxdestination,mapidx2origin,mapidx2destination